In [47]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import libpysal
import esda

import meshio

from scipy import stats
from scripts import morphoGrid as morph

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Extract required information

First we get the elevation from `badlands` output for a given time step

In [39]:
step = 700
dx = 5000.
folder = 'lem-models/Sarr/h5'

badland_topo = morph.morphoGrid(folder=folder, bbox = None, dx=dx)
badland_topo.loadHDF5(timestep=step)

# Regular grid coordinates
xyi = np.dstack([badland_topo.x.flatten(), badland_topo.y.flatten()])[0]
vertices = badland_topo.vertices
XY = vertices[:,:2]
elev = vertices[:,-1] - badland_topo.sl

Then, like for previous notebooks, we find the Sunda Shelf coordinates: 

In [40]:
shelfXYZ = "data/sundaland_shelf_coordinates.csv"
pd_shelf = pd.read_csv(
            shelfXYZ,
            sep=r"	",
            engine="c",
            na_filter=False,
            dtype=np.float,
            low_memory=False,
        )

shelfXY = pd_shelf.values[:, 0:2]
shelfZ = pd_shelf.values[:, 2]
tree = cKDTree(shelfXY)
dists, inds = tree.query(XY, k=1)
shelfIDs = np.where(np.logical_and(dists<=5000, elev>0.))[0]

Finally we specify the circuitscape `ASC` output file.

In [41]:
circuitscape  = 'cost/Sarr_700_corridor_cum_curmap.asc'

## Compute z-score

To normalize connectivity over time, we will compute current flow z-score values. 

In [42]:
def result_current_zscore(file):
    
    curmap1  = np.flipud(np.loadtxt(file, skiprows=6))

    iddd = np.where(curmap1.flatten()>0)[0]

    posflow = curmap1.flatten()[iddd]
    vals = stats.zscore(posflow, axis=0) #, ddof=1, nan_policy='raise')

    tree = cKDTree(xyi) 
    res1 = curmap1.flatten()

    # On the uniform grid
    distances, indices = tree.query(XY, k=3)

    onIDs = np.where(distances[:, 0] == 0)[0]
    inIDs = np.where(distances[:, 0] > 0)[0]

    weights = np.ones(distances.shape)
    weights[inIDs,:] = 1.0 / distances[inIDs,:] ** 2

    denum = 1.0 / np.sum(weights, axis=1)

    resmap1 = np.sum(weights * res1[indices], axis=1) * denum

    if len(onIDs) > 0:
        resmap1[onIDs] = res1[indices[onIDs, 0]]
    

    tree = cKDTree(xyi[iddd]) 
    score = vals

    # On the uniform grid
    distances, indices = tree.query(XY, k=3)

    onIDs = np.where(distances[:, 0] == 0)[0]
    inIDs = np.where(distances[:, 0] > 0)[0]

    weights = np.ones(distances.shape)
    weights[inIDs,:] = 1.0 / distances[inIDs,:] ** 2

    denum = 1.0 / np.sum(weights, axis=1)

    score1 = np.sum(weights * score[indices], axis=1) * denum

    if len(onIDs) > 0:
        score1[onIDs] = score[indices[onIDs, 0]]


    return resmap1, score1

In [43]:
current, zscore = result_current_zscore(circuitscape)

## Getis-Ord index

We also compute the Getis-Ord Gi* index showing statistically significant connectivity hotspots. 

In [44]:
def result_gis(res):
    
    kd = libpysal.cg.KDTree(np.array(vertices[shelfIDs,:2]))
    weights = libpysal.weights.KNN(kd,12)

    flow = np.array(res[shelfIDs])

    # Gi*
    gostars = esda.getisord.G_Local(flow, weights, star=True)

    GIs = np.zeros(len(vertices))
    GIstar = np.zeros(len(shelfIDs))

    # Break observations into significant or not
    sig = gostars.p_sim < 0.05
    # Plot non-significant clusters
    GIstar[(sig==True)] = 0
    # Plot HH clusters
    GIstar[(gostars.Zs > 0) & (sig==True)] = 5
    # Plot LL clusters
    GIstar[(gostars.Zs < 0) & (sig==True)] = -5
    GIs[shelfIDs] = GIstar
    
    GIs[GIs>0] = 1
    GIs[GIs<0] = -1
        
    idpos = np.where(GIs == 1)[0]
    idneg = np.where(GIs == -1)[0]

    combGIs = np.zeros(len(GIs))
    combGIs[idpos] = 1
    combGIs[idneg] = -1

    idzeros = np.where(combGIs == 0)[0]
    XY = vertices[:,:2] 
    tin_tree = cKDTree(XY)

    dist, ids = tin_tree.query(XY[idzeros], k=7)
    surrounding = np.sum(combGIs[ids],axis=1)

    sumsur = np.zeros(len(surrounding))
    sumsur[surrounding>2] = 1
    sumsur[surrounding<-2] = -1

    combGIs[idzeros] = sumsur

    return combGIs

In [45]:
GIstar = result_gis(current)

## Output files

We now create a `VTK` output containing all connectivity parameters.

In [62]:
# Get costs 
costmesh = 'vtk/Sarr700.corridor.cost.vtk'
mesh = meshio.read(costmesh)

vertices = mesh.points
cells = mesh.cells_dict['triangle']
elev = mesh.point_data['Z']
ED = mesh.point_data['ED']
logFA = mesh.point_data['FA']
lec = mesh.point_data['costLEC']
river_corridor = mesh.point_data['costRiver']
cost_elev = mesh.point_data['costElev']
totcost = mesh.point_data['all']
zscore[elev<0] = 0.

The `VTK` file can be visualised in Paraview.

In [63]:
vtkfile = 'vtk/Sarr700.corridor.bio.vtk'
vis_mesh = meshio.Mesh(vertices, {'triangle': cells}, 
                       point_data={"Z":elev, "ED":ED, 
                       "costLEC":lec, 'FA':logFA, 'costRiver':river_corridor,
                       "costElev":cost_elev,"totcost":totcost,
                       "zscore":zscore,"current":current,"GIstar":GIstar,
                      })
meshio.write(vtkfile, vis_mesh)

<div align="center">
    <img width=1000 src="img/connectivity_params.png?raw=true" alt="Badlands output" title="Badlands output"</img>
</div>

